# Challenge - Tweetons !

Nous allons effectuer les mêmes missions que dans la quête précédente, afin de comparer les deux méthodes (TfIdf Vectorizer et CountVectorizer). Pour rappel, les missions étaient les suivantes :

1. Importe **cet ensemble de données** de tweets dans un DataFrame.

2. Ne garde que les tweets positifs et négatifs (tu excluras donc les `neutral`). Quel est le pourcentage de tweets positifs/négatifs ?

3. Copie la colonne `text` dans une **Série** `X`, et la colonne `sentiment` dans une **Série** `y`. Applique un train test split avec le `random_state = 32`.

4. Crée un modèle `vectorizer` avec scikit-learn en utilisant la méthode `TfidfVectorizer`. Entraîne ton modèle sur `X_train`, puis crée une matrice de *features* `X_train_CV`. Crée la matrice `X_test_CV` sans ré-entraîner le modèle. Le format de la matrice `X_test_CV` doit être `4091x15806` avec `44633 stored elements`.

5. Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : `0.932` pour le test d'entraînement, et `0.873` pour l'ensemble de test.

In [2]:
import pandas as pd
df = pd.read_csv('data/train.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [3]:
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]
positive_percentage = (df_filtered['sentiment'] == 'positive').mean() * 100
negative_percentage = (df_filtered['sentiment'] == 'negative').mean() * 100

print("Pourcentage de tweets positifs :",positive_percentage, "%")
print("Pourcentage de tweets négatifs :",negative_percentage, "%")


Pourcentage de tweets positifs : 52.44759518425717 %
Pourcentage de tweets négatifs : 47.55240481574283 %


In [4]:
from sklearn.model_selection import train_test_split

X = df_filtered['text']
y = df_filtered['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_CV = vectorizer.fit_transform(X_train)
X_test_CV = vectorizer.transform(X_test)


print(f"Forme de X_train_CV : {X_train_CV.shape}")
print(f"Nombre d'éléments stockés dans X_train_CV: {X_train_CV.nnz}")
print("--------------")
print(f"Forme de X_test_CV : {X_test_CV.shape}")
print(f"Nombre d'éléments stockés dans X_test_CV: {X_test_CV.nnz}")

Forme de X_train_CV : (12272, 15806)
Nombre d'éléments stockés dans X_train_CV: 144578
--------------
Forme de X_test_CV : (4091, 15806)
Nombre d'éléments stockés dans X_test_CV: 44633


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)
model.fit(X_train_CV, y_train)
y_train_pred = model.predict(X_train_CV)
y_test_pred = model.predict(X_test_CV)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Exactitude sur l'ensemble d'entraînement : {train_accuracy:.3f}")
print(f"Exactitude sur l'ensemble de test : {test_accuracy:.3f}")


Exactitude sur l'ensemble d'entraînement : 0.931
Exactitude sur l'ensemble de test : 0.873


In [8]:
errors = (y_test != y_test_pred)
incorrect_indices = y_test[errors].index
incorrect_tweets = X_test[errors]

print("Tweets mal classifiés :")
for i in range(min(10, len(incorrect_tweets))):
    tweet = incorrect_tweets.iloc[i]
    idx = incorrect_indices[i]
    
    real_sentiment = y_test.loc[idx]
    predicted_sentiment = y_test_pred[y_test.index.get_loc(idx)]
    
    # Affiche le tweet, la prédiction et la classe réelle
    print(f"\nTweet : {tweet}")
    print(f"Prédiction : {'Positif' if predicted_sentiment == 'positive' else 'Négatif'}")
    print(f"Réel : {'Positif' if real_sentiment == 'positive' else 'Négatif'}")


Tweets mal classifiés :

Tweet : HollowbabesHere comes the utter shite #bgt <I completely agree
Prédiction : Positif
Réel : Négatif

Tweet :  SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx
Prédiction : Positif
Réel : Négatif

Tweet : i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could  somebody could send me a link? thaank you <3
Prédiction : Négatif
Réel : Positif

Tweet : I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much
Prédiction : Positif
Réel : Négatif

Tweet : I can only message those who message me, if we`re fwends...so those that want replies..follow me.  hmm..that sounds funny..
Prédiction : Négatif
Réel : Positif

Tweet : wish I could feel no pain (8)  but it`s ok, at least they like Brazil!
Prédiction : Négatif
Réel : Positif

Tweet :  so glad i`m not at uni anymore
Prédiction : Négati